In [93]:
GYRO = False
SEQUENCE_LENGTH = 5
SEQUENCE_OVERLAP = 2
BATCH_SIZE = 10
EPOCHS = 30
CNN = True
MODEL_NAME = f"AllInOne_CNN:{CNN}_pochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 7
TEST_SIZE = 7
LEAVE_ONE_OUT = False
NORMALISE = True
OVERLAP_ON_TEST_SET = False
TRANSFORM = True
AUGMENT = False
AUGMENT_SIZE = 2 # 1 means 1 extra augmented sample per sample, 2 means 2 extra augmented samples per sample etc.
AUGMENT_NOISE = 0.03
DIFFERENCES = True
INCLUDE_OTHER = True
DERIVATIVES = True
EARLY_STOPPING = False
RANDOM_TEST_SET = False

### This code takes in a position, eg "sitting_or_standing", "lying_down_left" etc and trains a model for just classifying activities of that position

In [94]:
import helpers.file_tagger as file_tagger
import helpers.sequence_generator as sequence_generator
import helpers.split_by_student as split_by_student
from keras.callbacks import EarlyStopping


import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models, regularizers
from sklearn.model_selection import train_test_split
import time


POSSIBLE_ACTIVITIES = [
    "sitting&coughing",
    "sitting&hyperventilating",
    "sitting&normal_breathing",
    
    "standing&coughing",
    "standing&hyperventilating",
    "standing&normal_breathing",
    
    "sitting&talking",
    "sitting&eating",
    "sitting&singing",
    "sitting&laughing",

    "standing&talking",
    "standing&eating",
    "standing&singing",
    "standing&laughing",
    "lying_down_left&normal_breathing", 
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&talking",
    "lying_down_left&singing",
    "lying_down_left&laughing",
    "lying_down_right&normal_breathing",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&talking",
    "lying_down_right&singing",
    "lying_down_right&laughing",
    "lying_down_back&normal_breathing",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&talking",
    "lying_down_back&singing",
    "lying_down_back&laughing",
    "lying_down_stomach&normal_breathing",
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&talking",
    "lying_down_stomach&singing",
    "lying_down_stomach&laughing",

]

POSSIBLE_OUTCOMES= [
    "sitting_or_standing&normal_breathing",
    "sitting_or_standing&coughing",
    "sitting_or_standing&hyperventilating",
    "sitting_or_standing&other",
    
    "lying_down_left&normal_breathing",
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&other",
    
    "lying_down_right&normal_breathing",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&other",
    
    "lying_down_back&normal_breathing",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&other",
    
    "lying_down_stomach&normal_breathing",
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&other",
    
]

if not INCLUDE_OTHER:
    POSSIBLE_OUTCOMES= [
    "sitting_or_standing&normal_breathing",
    "sitting_or_standing&coughing",
    "sitting_or_standing&hyperventilating",
    "sitting_or_standing&talking",
    "sitting_or_standing&singing",
    "sitting_or_standing&laughing",
    "sitting_or_standing&eating",
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&talking",
    "lying_down_left&singing",
    "lying_down_left&laughing",
    "lying_down_left&normal_breathing",
    "lying_down_right&normal_breathing",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&talking",
    "lying_down_right&singing",
    "lying_down_right&laughing",
    "lying_down_back&normal_breathing",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&talking",
    "lying_down_back&singing",
    "lying_down_back&laughing",
    "lying_down_stomach&normal_breathing",
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&talking",
    "lying_down_stomach&singing",
    "lying_down_stomach&laughing"]

DATA_DIRECTORY = "./all_respeck"
LABEL_TO_INDEX = {label: idx for idx, label in enumerate(POSSIBLE_OUTCOMES)}

if OVERLAP_ON_TEST_SET:
    TEST_SEQUENCE_OVERLAP = SEQUENCE_OVERLAP
else:
    TEST_SEQUENCE_OVERLAP = 0

In [95]:
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in POSSIBLE_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if csv_file in file_names:
                if gyro:
                    sequences = sequence_generator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)
                else:
                    sequences = sequence_generator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)

                # iterate through each generated sequence
                for sequence in sequences:
                    position = key.split("&")[0]
                    activity = key.split("&")[1]

                    if INCLUDE_OTHER and (activity == "talking" or activity == "singing" or activity == "laughing" or activity == "eating"):
                        activity = "other"

                    if position == "standing" or position == "sitting":
                        position = "sitting_or_standing"
                        
                    tagged_data.append((position + "&" + activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")

    return tagged_data

In [96]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    print("Training CNN model")
    if GYRO:
        width = 6
    else:
        width = 3
    if TRANSFORM:
        width = width+3
    if DIFFERENCES:
        width = width+3
    if DERIVATIVES:
        width = width+3

    print("Input data shape: ", input_data.shape)
    
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),


        
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),

        #layers.Dropout(0.1),

        
        layers.Conv1D(128, 3, activation='relu'), 
        layers.MaxPooling1D(2),

        layers.Dropout(0.6),
       
        layers.Flatten(),
        
        layers.Dense(128, activation='relu', activity_regularizer=regularizers.l2(0.1)),
        layers.Dense(64, activation='relu'), 
        #layers.Dense(64, activation='relu'),
        #layers.Dense(32, activation='relu'),
        #layers.Dense(64, activation='relu', activity_regularizer=regularizers.l2(0.0001)), 
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   # Train the CNN model
    if not EARLY_STOPPING:
        if len(validation_data[0]) == 0:
            model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
        else:
            model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)
    else:
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='max', restore_best_weights=True)
        if len(validation_data[0]) == 0:
            model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])
        else:
            model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data, callbacks=[early_stopping])

    return model

In [97]:
def train_model_LSTM(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    print("\n\n ===== Training LSTM Model =====\n")
    if GYRO:
        width = 6
    else:
        width = 3
    if TRANSFORM:
        width = width+3
    if DIFFERENCES:
        width = width+3
    if DERIVATIVES:
        width = width+3

    print("Input data shape: ", input_data.shape)
    
    model = Sequential([
        
        layers.LSTM(64, activation='relu', return_sequences=True, input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        
        layers.LSTM(64, return_sequences=True, activation='relu'),
        layers.LSTM(128, return_sequences=True, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        layers.Flatten(),
        layers.Dense(128, activation='relu', activity_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [98]:
def augment_data(sequences, labels, num_augmentations):
    augmented_sequences = []
    augmented_labels = []
    
    for sequence, label in zip(sequences, labels):
        for _ in range(num_augmentations):
            augmented_sequence = sequence + np.random.normal(0, AUGMENT_SIZE, size=sequence.shape)
            augmented_sequences.append(augmented_sequence)
            augmented_labels.append(label)
    
    return augmented_sequences, augmented_labels


In [99]:
def fft(data):

    # Extract x, y, and z data
    x_data = data[:, 0]
    y_data = data[:, 1]
    z_data = data[:, 2]

    # Apply FFT to each axis
    x_fft = np.fft.fft(x_data)
    y_fft = np.fft.fft(y_data)
    z_fft = np.fft.fft(z_data)

    # The result is complex numbers, so you may want to take the magnitude
    x_magnitude = np.abs(x_fft)
    y_magnitude = np.abs(y_fft)
    z_magnitude = np.abs(z_fft)

    # If needed, you can also compute the corresponding frequencies
    # The frequencies are in cycles per time unit (usually, Hz if your time unit is seconds)
    x_frequencies = np.fft.fftfreq(len(x_data))
    y_frequencies = np.fft.fftfreq(len(y_data))
    z_frequencies = np.fft.fftfreq(len(z_data))

    representation = []
    for i in range(len(x_magnitude)):
        representation.append([x_magnitude[i], y_magnitude[i], z_magnitude[i]]) #, x_frequencies[i], y_frequencies[i], z_frequencies[i]])

    return representation

def extract_fft(train_data, dev_data, test_data):
    train_features = [fft(sequence) for sequence in train_data]
    dev_features = [fft(sequence) for sequence in dev_data]
    test_features = [fft(sequence) for sequence in test_data]

    return train_features, dev_features, test_features


def merge_arrays(arr1, arr2):
    return np.concatenate((arr1, arr2), axis=1)

#def normalise(sequence):
#    """
#    Normalizes a matrix of accelerometer values.
#    """
#    norm = np.linalg.norm(sequence, axis=1)
#    norm[norm == 0] = 1
#    return sequence / norm[:, np.newaxis]

def differential(data):
    # Extract x, y, and z data
    x_data = data[:, 0]
    y_data = data[:, 1]
    z_data = data[:, 2]

    # Compute the differences between consecutive data points
    x_diff = np.diff(x_data)
    y_diff = np.diff(y_data)
    z_diff = np.diff(z_data)

    # Add a 0 at the start of the differential variables
    x_diff = np.insert(x_diff, 0, 0)
    y_diff = np.insert(y_diff, 0, 0)
    z_diff = np.insert(z_diff, 0, 0)

    # Combine the differential values into a representation
    representation = []
    for i in range(len(x_diff)):
        representation.append([x_diff[i], y_diff[i], z_diff[i]])

    return representation

def extract_differentials(train_data, dev_data, test_data):
    train_features = [differential(sequence) for sequence in train_data]
    dev_features = [differential(sequence) for sequence in dev_data]
    test_features = [differential(sequence) for sequence in test_data]

    return train_features, dev_features, test_features

def derivative(data):
    # Extract x, y, and z data
    x_data = data[:, 0]
    y_data = data[:, 1]
    z_data = data[:, 2]

    # Compute the derivative of the data
    x_derivative = np.gradient(x_data)
    y_derivative = np.gradient(y_data)
    z_derivative = np.gradient(z_data)

    # Combine the derivative values into a representation
    representation = []
    for i in range(len(x_derivative)):
        representation.append([x_derivative[i], y_derivative[i], z_derivative[i]])

    return representation

def extract_gradients(train_data, dev_data, test_data):
    train_features = [derivative(sequence) for sequence in train_data]
    dev_features = [derivative(sequence) for sequence in dev_data]
    test_features = [derivative(sequence) for sequence in test_data]

    return train_features, dev_features, test_features

In [100]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size, randomize=RANDOM_TEST_SET)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_SEQUENCE_OVERLAP, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_SEQUENCE_OVERLAP, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [101]:
def leave_one_out(isCNN=CNN):
    students = split_by_student.get_list_of_stutents()
    test_accuracies = []
    for test_student in students:
        print("testing student: " + str(test_student))
        print("training students: " + str([student for student in students if student != test_student]))
        
        test_files, training_files = split_by_student.get_list_of_files(test_student)

        tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
        tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_SEQUENCE_OVERLAP, file_names=test_files)

        train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
        test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

        if AUGMENT:
            augmented_train_data, augmented_train_labels = augment_data(train_data, train_labels, AUGMENT_SIZE)
            train_data = np.concatenate((train_data, augmented_train_data))
            train_labels = np.concatenate((train_labels, augmented_train_labels))
            print("length of training data with augmentation: " + str(len(train_data)))

        if TRANSFORM:
            train_transform, dev_transform, test_transform = extract_fft(train_data, [], test_data)
            train_data = np.array(train_data)
            train_transform = np.array(train_transform)
           
            test_data = np.array(test_data)
            test_transform = np.array(test_transform)

            train_data = np.array([merge_arrays(train_data[i], train_transform[i]) for i in range(len(train_data))])
            test_data = np.array([merge_arrays(test_data[i], test_transform[i]) for i in range(len(test_data))])

        if DIFFERENCES:
            train_differential, dev_differential, test_differential = extract_differentials(train_data, [], test_data)
            train_data = np.array(train_data)
            train_differential = np.array(train_differential)
           
            test_data = np.array(test_data)
            test_differential = np.array(test_differential)

            train_data = np.array([merge_arrays(train_data[i], train_differential[i]) for i in range(len(train_data))])
            test_data = np.array([merge_arrays(test_data[i], test_differential[i]) for i in range(len(test_data))])

        
        if DERIVATIVES:
            train_derivatives, dev_derivatives, test_derivatives = extract_gradients(train_data, [], test_data)
            train_data = np.array(train_data)
            train_derivatives = np.array(train_derivatives)
         
            test_data = np.array(test_data)
            test_derivatives = np.array(test_derivatives)

            train_data = np.array([merge_arrays(train_data[i], train_derivatives[i]) for i in range(len(train_data))])
            test_data = np.array([merge_arrays(test_data[i], test_derivatives[i]) for i in range(len(test_data))])


        if isCNN:
            model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(test_data, test_labels))
        else:
            model = train_model_LSTM(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(test_data, test_labels))
        
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        test_accuracies.append(test_accuracy)
        print("for student " + str(test_student) + " the accuracy is " + str(test_accuracy))
        print("average accuracy so far: " + str(sum(test_accuracies)/len(test_accuracies)))
        print("number of students tested so far: " + str(len(test_accuracies)))
        time.sleep(3)

        
    print("Accuracy for each student:")
    print(", ".join([f"{student}: {accuracy}" for student, accuracy in zip(students, test_accuracies)]))
    print("Average overall accuracy:", sum(test_accuracies)/len(test_accuracies))



In [102]:
if LEAVE_ONE_OUT:
    leave_one_out()
    

if not LEAVE_ONE_OUT:
    train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE)

    if AUGMENT:
        augmented_train_data, augmented_train_labels = augment_data(train_data, train_labels, AUGMENT_SIZE)
        train_data = np.concatenate((train_data, augmented_train_data))
        train_labels = np.concatenate((train_labels, augmented_train_labels))
        print("length of training data with augmentation: " + str(len(train_data)))

    if TRANSFORM:
        train_transform, dev_transform, test_transform = extract_fft(train_data, dev_data, test_data)
        train_data = np.array(train_data)
        train_transform = np.array(train_transform)
        dev_data = np.array(dev_data)
        dev_transform = np.array(dev_transform)
        test_data = np.array(test_data)
        test_transform = np.array(test_transform)

        train_data = np.array([merge_arrays(train_data[i], train_transform[i]) for i in range(len(train_data))])
        dev_data = np.array([merge_arrays(dev_data[i], dev_transform[i]) for i in range(len(dev_data))])
        test_data = np.array([merge_arrays(test_data[i], test_transform[i]) for i in range(len(test_data))])

    if DIFFERENCES:
        train_differential, dev_differential, test_differential = extract_differentials(train_data, dev_data, test_data)
        train_data = np.array(train_data)
        train_differential = np.array(train_differential)
        dev_data = np.array(dev_data)
        dev_differential = np.array(dev_differential)
        test_data = np.array(test_data)
        test_differential = np.array(test_differential)

        train_data = np.array([merge_arrays(train_data[i], train_differential[i]) for i in range(len(train_data))])
        dev_data = np.array([merge_arrays(dev_data[i], dev_differential[i]) for i in range(len(dev_data))])
        test_data = np.array([merge_arrays(test_data[i], test_differential[i]) for i in range(len(test_data))])

    if DERIVATIVES:
        train_derivatives, dev_derivatives, test_derivatives = extract_gradients(train_data, dev_data, test_data)
        train_data = np.array(train_data)
        train_derivatives = np.array(train_derivatives)
        dev_data = np.array(dev_data)
        dev_derivatives = np.array(dev_derivatives)
        test_data = np.array(test_data)
        test_derivatives = np.array(test_derivatives)

        train_data = np.array([merge_arrays(train_data[i], train_derivatives[i]) for i in range(len(train_data))])
        dev_data = np.array([merge_arrays(dev_data[i], dev_derivatives[i]) for i in range(len(dev_data))])
        test_data = np.array([merge_arrays(test_data[i], test_derivatives[i]) for i in range(len(test_data))])

    if CNN:
        model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) 
    else:
        model = train_model_LSTM(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) 


    # Evaluate the model on the test set
    if len(test_data) > 0:
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Train Set: s80, s87, s22, s3, s29, s12, s100, s17, s86, s5, s13, s66, s54, s39, s34, s42, s44, s48, s18, s55, s88, s38, s95, s50, s71, s75, s32, s46, s70, s30, s59, s74, s52, s33, s92, s97, s35, s8, s63, s72, s84, s57, s65, s11, s7, s67, s91, s15, s9, s79, s77, s43, s40, s1, s36, s61, s64
Dev Set: s21, s82, s51, s60, s98, s56, s102
Test Set: s45, s23, s83, s27, s93, s16, s96
there are 19268 tagged sequences in the dataset
there are 1592 tagged sequences in the dataset
there are 1591 tagged sequences in the dataset
Training CNN model
Input data shape:  (19268, 125, 12)
Epoch 1/30
1927/1927 [==============================] - 7s 4ms/step - loss: 1.3856 - accuracy: 0.5206 - val_loss: 0.9672 - val_accuracy: 0.6018
Epoch 2/30
1927/1927 [==============================] - 6s 3ms/step - loss: 0.9659 - accuracy: 0.6259 - val_loss: 0.8269 - val_accuracy: 0.6790
Epoch 3/30
1927/1927 [==============================] - 6s 3ms/step - loss: 0.8605 - accuracy: 0.6661 - val_loss: 0.8137 - val_accuracy: 

In [103]:
# Save the trained model
#model.save("submission_folder/Models/task3_model_NOTFINAL.keras")

In [104]:


#tflite_model_path = "models/presentation_models/task3_model_final.tflite"
#interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
#interpreter.allocate_tensors()

    # Get input and output details
#input_details = interpreter.get_input_details()
#output_details = interpreter.get_output_details()

    # Print model summary
#print("TFLite Model Summary:")
#print("Input Shape:", input_details[0]['shape'])
##print("Input Type:", input_details[0]['dtype'])
#print("Output Shape:", output_details[0]['shape'])
#print("Output Type:", output_details[0]['dtype'])



